In [1]:
# Requires brainvisa

In [1]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt
import os

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from soma import aims
import json
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np
import glob

In [3]:
a = anatomist.Anatomist()

create qapp
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module meshsplit
loading module palettecontrols
loading module volumepalettes


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [4]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

Lspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lspam_model_meshes_1.arg"

json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

path_summary = "/neurospin/grip/protocols/MRI/NR2F1_GD_2025/results/DeepFolding"
path_file = "all_regions.csv"


In [5]:
with open(json_regions) as f:
    regions = json.load(f)

In [6]:
file_to_display = f"{path_summary}/{path_file}"
print(file_to_display)

/neurospin/grip/protocols/MRI/NR2F1_GD_2025/results/DeepFolding/all_regions.csv


In [7]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [8]:
df = pd.read_csv(file_to_display)[["region", "p"]]
df["p"] = -np.log10(df["p"])
df.head()

,region,p
0,FIP_right,4.275724
1,SFinter-SFsup_left,0.026872
2,SFint-SR_right,0.420216
3,SC-SPeC_right,0.481486
4,SFmarginal-SFinfant_left,0.568636


In [9]:
res = df.groupby(['region']).mean()
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [10]:
res = res[~res.region.str.contains("fronto-parietal")]
res = res[~res.region.str.contains("STs-SGSM")]
res = res[~res.region.str.contains("FCLp-SGSM")]
res = res[~res.region.str.contains("CINGULATE")]

In [11]:
"CINGULATE_left".replace("CINGULATE", "CINGULATE.")

'CINGULATE._left'

In [12]:
def get_sulci(region):
    region = region.replace("CINGULATE", "CINGULATE.")
#     region = region.replace("S.Or", "S.Or.")
    list_sulci = list(regions['brain'][f"{region}"].keys())
    list_sulci = [x.replace("paracingular.", "S.F.int.") for x in list_sulci]
    return list_sulci

In [13]:
get_sulci("CINGULATE_left")

['S.F.int._left', 'F.C.M.ant._left']

In [14]:
# res["region"].tolist()

In [15]:
dic = {
    "FCLp-subsc-FCLa-INSULA": "F.C.L.p.-subsc.-F.C.L.a.-INSULA.",
    "FCMpost-SpC": "F.C.M.post.-S.p.C.",
    "FColl-SRh": "F.Coll.-S.Rh.",
    "FIP": "F.I.P.",
    "FPO-SCu-ScCal": "F.P.O.-S.Cu.-Sc.Cal.",
    "Lobule_parietal_sup": "Lobule_parietal_sup.",
    "SOr": "S.Or.",
    "SC-SPeC": "S.C.-S.Pe.C.",
    "SC-SPoC": "S.C.-S.Po.C.",
    "SC-sylv": "S.C.-sylv.",
    "SFinf-BROCA-SPeCinf": "S.F.inf.-BROCA-S.Pe.C.inf.",
    "SFint-FCMant": "S.F.int.-F.C.M.ant.",
    "SFint-SR": "S.F.int.-S.R.",
    "SFinter-SFsup": "S.F.inter.-S.F.sup.",
    "SFmarginal-SFinfant":"S.F.marginal-S.F.inf.ant.",
    "SFmedian-SFpoltr-SFsup": "S.F.median-S.F.pol.tr.-S.F.sup.",
    "S.Or.-SOlf": "S.Or.-S.Olf.",
    "SPeC": "S.Pe.C.",
    "SPoC": "S.Po.C.",
    "STi-SOTlat": "S.T.i.-S.O.T.lat.",
    "STi-STs-STpol": "S.T.i.-S.T.s.-S.T.pol.",
    "STsbr": "S.T.s.br.",
    "STs": "S.T.s.",
    "ScCal-SLi": "Sc.Cal.-S.Li.",
    "SsP-SPaint": "S.s.P.-S.Pa.int.",
}

In [16]:
for key in dic.keys():
    res["region"] = res["region"].str.replace(key, dic[key])
# res.head()
print(res)

                                    region         p   side
2    F.C.L.p.-subsc.-F.C.L.a.-INSULA._left  3.119186   left
3   F.C.L.p.-subsc.-F.C.L.a.-INSULA._right  2.568636  right
4                  F.C.M.post.-S.p.C._left  4.193820   left
5                 F.C.M.post.-S.p.C._right  4.356547  right
6                       F.Coll.-S.Rh._left  4.119186   left
7                      F.Coll.-S.Rh._right  4.958607  right
8                              F.I.P._left  3.744727   left
9                             F.I.P._right  4.275724  right
10               F.P.O.-S.Cu.-Sc.Cal._left  6.455932   left
11              F.P.O.-S.Cu.-Sc.Cal._right  3.886057  right
14               Lobule_parietal_sup._left  3.886057   left
15              Lobule_parietal_sup._right  6.055517  right
16                          OCCIPITAL_left  4.958607   left
17                         OCCIPITAL_right  4.698970  right
18                       S.C.-S.Pe.C._left  1.657577   left
19                      S.C.-S.Pe.C._rig

/tmp/ipykernel_11428/3221059896.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  res["region"] = res["region"].str.replace(key, dic[key])


In [17]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
# res.head()
print(res)

                                    region         p   side  \
2    F.C.L.p.-subsc.-F.C.L.a.-INSULA._left  3.119186   left   
3   F.C.L.p.-subsc.-F.C.L.a.-INSULA._right  2.568636  right   
4                  F.C.M.post.-S.p.C._left  4.193820   left   
5                 F.C.M.post.-S.p.C._right  4.356547  right   
6                       F.Coll.-S.Rh._left  4.119186   left   
7                      F.Coll.-S.Rh._right  4.958607  right   
8                              F.I.P._left  3.744727   left   
9                             F.I.P._right  4.275724  right   
10               F.P.O.-S.Cu.-Sc.Cal._left  6.455932   left   
11              F.P.O.-S.Cu.-Sc.Cal._right  3.886057  right   
14               Lobule_parietal_sup._left  3.886057   left   
15              Lobule_parietal_sup._right  6.055517  right   
16                          OCCIPITAL_left  4.958607   left   
17                         OCCIPITAL_right  4.698970  right   
18                       S.C.-S.Pe.C._left  1.657577   

In [18]:
res = res.sort_values(by="p", ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

,region,p,side,sulcus
48,S.T.s._left,5.161151,left,S.T.s._left
46,S.T.i.-S.T.s.-S.T.pol._left,4.602060,left,S.T.i.ant._left
46,S.T.i.-S.T.s.-S.T.pol._left,4.602060,left,S.T.i.post._left
46,S.T.i.-S.T.s.-S.T.pol._left,4.602060,left,S.T.s._left
46,S.T.i.-S.T.s.-S.T.pol._left,4.602060,left,S.T.pol._left
51,S.T.s.br._right,4.119186,right,F.I.P.r.int.1_right
51,S.T.s.br._right,4.119186,right,F.I.P.r.int.2_right
51,S.T.s.br._right,4.119186,right,S.T.s.ter.asc.ant._right
51,S.T.s.br._right,4.119186,right,S.T.s.ter.asc.post._right
51,S.T.s.br._right,4.119186,right,S.GSM._right


In [19]:
for _, row in res.iterrows():
    print(row.sulcus)

S.s.P._right
S.Pa.int._right
F.P.O._left
S.Pa.t._left
S.Cu._left
F.Cal.ant.-Sc.Cal._left
S.O.p._left
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.Li.ant._right
S.Li.post._right
F.I.P._right
S.Po.C.sup._right
S.Pa.sup._right
S.T.i.ant._left
S.T.i.post._left
S.O.T.lat.ant._left
S.O.T.lat.int._left
S.O.T.lat.med._left
S.O.T.lat.post._left
S.T.i.ant._right
S.T.i.post._right
S.O.T.lat.ant._right
S.O.T.lat.int._right
S.O.T.lat.med._right
S.O.T.lat.post._right
S.T.s._left
OCCIPITAL_left
F.Coll._right
S.Rh._right
OCCIPITAL_right
S.T.i.ant._left
S.T.i.post._left
S.T.s._left
S.T.pol._left
F.C.M.post._right
S.C.LPC._right
S.p.C._right
F.I.P._right
F.C.M.post._left
S.C.LPC._left
S.p.C._left
F.Coll._left
S.Rh._left
F.I.P.r.int.1_right
F.I.P.r.int.2_right
S.T.s.ter.asc.ant._right
S.T.s.ter.asc.post._right
S.GSM._right
S.s.P._left
S.Pa.int._left
F.P.O._right
S.Pa.t._right
S.Cu._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
F.I.P._left
S.Po.C.sup._left
S.Pa.sup._left
F.I.P._left
S.Po.C.sup._left
F.I.P.Po.C.i

In [20]:
res[res.sulcus=="S.F.orbitaire._right"]

,region,p,side,sulcus
33,S.F.marginal-S.F.inf.ant._right,0.283997,right,S.F.orbitaire._right


In [21]:
res

,region,p,side,sulcus
55,S.s.P.-S.Pa.int._right,7.267606,right,S.s.P._right
55,S.s.P.-S.Pa.int._right,7.267606,right,S.Pa.int._right
10,F.P.O.-S.Cu.-Sc.Cal._left,6.455932,left,F.P.O._left
10,F.P.O.-S.Cu.-Sc.Cal._left,6.455932,left,S.Pa.t._left
10,F.P.O.-S.Cu.-Sc.Cal._left,6.455932,left,S.Cu._left
...,...,...,...,...
30,S.F.inter.-S.F.sup._left,0.026872,left,S.F.inter._left
30,S.F.inter.-S.F.sup._left,0.026872,left,S.F.sup._left
35,S.F.median-S.F.pol.tr.-S.F.sup._right,0.004365,right,S.F.median._right
35,S.F.median-S.F.pol.tr.-S.F.sup._right,0.004365,right,S.F.polaire.tr._right


In [24]:
res.p.max()

7.267606240177032

In [ ]:
def set_color_property(res, side):
    global dic

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex['p_value'] = 0.

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                    vertex['p_value'] = max(vertex['p_value'],
                                                row.p)
                    # if row.p < -np.log10(0.05/56):
                    #     vertex['p_value'] = 0.
                    # elif vertex['p_value'] != 0.: 
                    #     vertex['p_value'] = min(vertex['p_value'],
                    #                             row.p)
                    # else:
                    #     vertex['p_value'] = row.p
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('p_value')
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])
    dic[f"ana{side}"].setPalette("green_yellow_red",
                              minVal=-np.log10(0.05), maxVal=8,
                              absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)

middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)

In [23]:
dic = {}

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)

Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60


Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout
